In [ ]:
!pip install transformers torch torchtext pandas datasets -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 7.7 MB/s eta 0:00:00


In [ ]:
from datasets import load_dataset
import pandas as pd

In [ ]:
symptoms = load_dataset("QuyenAnhDE/Diseases_Symptoms")
symptoms

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Repo card metadata block was not found. Setting CardData to empty.


Generating train split:   0%|          | 0/400 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['Code', 'Name', 'Symptoms', 'Treatments'],
        num_rows: 400
    })
})

In [ ]:
data = [{"Name": item['Name'], "Symptoms": item['Symptoms']} for item in symptoms['train']]
df = pd.DataFrame(data)
df.head()

,Name,Symptoms
0,Panic disorder,"Palpitations, Sweating, Trembling, Shortness o..."
1,Vocal cord polyp,"Hoarseness, Vocal Changes, Vocal Fatigue"
2,Turner syndrome,"Short stature, Gonadal dysgenesis, Webbed neck..."
3,Cryptorchidism,"Absence or undescended testicle(s), empty scro..."
4,Ethylene glycol poisoning-1,"Nausea, vomiting, abdominal pain, General mala..."


In [ ]:
df['Symptoms'] = df['Symptoms'].apply(lambda x: ", ".join(x.split(", ")))

In [ ]:
from transformers import GPT2Tokenizer
from transformers import GPT2LMHeadModel

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split

import time
from tqdm import tqdm

In [ ]:
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

device

device(type='cuda')

In [ ]:
model_name = "distilbert/distilgpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name).to(device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/353M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [ ]:
model

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-5): 6 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [ ]:
class LanguageModel(Dataset):
    def __init__(self, df, tokenizer):
        self.labels = df.columns
        self.tokenizer = tokenizer
        self.data = df.to_dict(orient='records')
        self.max_length = self.fittest_max_length(df)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, item):
        x = self.data[item]['Name']
        y = self.data[item]['Symptoms']
        text = f"{x} | {y}"
        tokens = self.tokenizer.encode_plus(text, return_tensors='pt', max_length=self.max_length, padding='max_length', truncation=True)
        return tokens

    def fittest_max_length(self, df):
        max_value = max(len(max(df['Name'], key=len)), len(max(df['Symptoms'], key=len)))
        x = 2
        while x < max_value: x = x * 2
        return x

In [ ]:
data_sample = LanguageModel(df, tokenizer)

In [ ]:
train_len = int(0.8 * len(data_sample))
valid_len = len(data_sample) - train_len
train_data, valid_data = random_split(data_sample, [train_len, valid_len])

In [ ]:
train_len, valid_len

(320, 80)

In [ ]:
BATCH_SIZE = 8

In [ ]:
train_loader = DataLoader(dataset=train_data, batch_size=BATCH_SIZE, shuffle=True)
valid_loader = DataLoader(dataset=valid_data, batch_size=BATCH_SIZE)

In [ ]:
criterion = nn.CrossEntropyLoss(ignore_index=tokenizer.pad_token_id)
optimizer = optim.Adam(params=model.parameters(), lr=5e-4)
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
gpu = 0
results = pd.DataFrame(columns=['Epoch', 'Transformers', 'Batch Size', 'GPU', 'Training Loss', 'Validation Loss', 'Epoch Duration Sec'])

In [ ]:
num_epochs = 10

In [ ]:
for epoch in range(num_epochs):
    start_time = time.time()
    model.train()
    epoch_training_loss = 0
    train_iterator = tqdm(iterable=train_loader, desc=f"Training Epoch: {epoch+1}/{num_epochs} Batch Size: {BATCH_SIZE}, Transformers: {model_name}")
    for batch in train_iterator:
        optimizer.zero_grad()
        inputs = batch['input_ids'].squeeze(1).to(device)
        targets = inputs.clone()
        outputs = model(input_ids=inputs, labels=targets)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        train_iterator.set_postfix({"Training Loss": loss.item()})
        epoch_training_loss += loss.item()
    avg_epoch_training_loss = epoch_training_loss / len(train_iterator)

    model.eval()
    epoch_valid_loss = 0
    total_loss = 0
    valid_iterator = tqdm(iterable=valid_loader, desc=f"Valid Epoch: {epoch+1}/{num_epochs}")
    with torch.no_grad():
        for batch in valid_iterator:
            inputs = batch['input_ids'].squeeze(1).to(device)
            targets = inputs.clone()
            outputs = model(input_ids=inputs, labels=targets)
            loss = outputs.loss
            total_loss += loss
            valid_iterator.set_postfix({"Valid Loss": loss.item()})
            epoch_valid_loss += loss.item()
    avg_epoch_valid_loss = epoch_valid_loss / len(valid_iterator)
    end_time = time.time()
    epoch_duration_time = start_time-end_time
    new_row = {
        "Epoch": epoch+1,
        "Transformers": model_name,
        'Batch Size': BATCH_SIZE,
        "GPU": gpu,
        'Training Loss': avg_epoch_training_loss,
        'Validation Loss': avg_epoch_valid_loss,
        'Epoch Duration Sec': epoch_duration_time
    }
    results.loc[len(results)] = new_row
    print(f"Epoch: {epoch+1} Validation Loss: {total_loss/len(valid_iterator)}")





Training Epoch: 1/10 Batch Size: 8, Transformers: distilbert/distilgpt2: 100%|██████████| 40/40 [00:32<00:00,  1.21it/s, Training Loss=0.235]
Valid Epoch: 1/10: 100%|██████████| 10/10 [00:02<00:00,  4.09it/s, Valid Loss=0.199]


Epoch: 1 Validation Loss: 0.19507227838039398


Training Epoch: 2/10 Batch Size: 8, Transformers: distilbert/distilgpt2: 100%|██████████| 40/40 [00:32<00:00,  1.22it/s, Training Loss=0.158]
Valid Epoch: 2/10: 100%|██████████| 10/10 [00:02<00:00,  3.86it/s, Valid Loss=0.163]


Epoch: 2 Validation Loss: 0.15704265236854553


Training Epoch: 3/10 Batch Size: 8, Transformers: distilbert/distilgpt2: 100%|██████████| 40/40 [00:35<00:00,  1.14it/s, Training Loss=0.153]
Valid Epoch: 3/10: 100%|██████████| 10/10 [00:02<00:00,  3.83it/s, Valid Loss=0.157]


...

Epoch: 8 Validation Loss: 0.19902129471302032


Training Epoch: 9/10 Batch Size: 8, Transformers: distilbert/distilgpt2: 100%|██████████| 40/40 [00:34<00:00,  1.16it/s, Training Loss=0.0413]
Valid Epoch: 9/10: 100%|██████████| 10/10 [00:02<00:00,  3.86it/s, Valid Loss=0.207]


Epoch: 9 Validation Loss: 0.20440326631069183


Training Epoch: 10/10 Batch Size: 8, Transformers: distilbert/distilgpt2: 100%|██████████| 40/40 [00:34<00:00,  1.16it/s, Training Loss=0.0297]
Valid Epoch: 10/10: 100%|██████████| 10/10 [00:02<00:00,  3.87it/s, Valid Loss=0.219]

Epoch: 10 Validation Loss: 0.21180544793605804


In [ ]:
input_str = "Kidney Failure"
input_ids = tokenizer.encode(input_str, return_tensors='pt').to(device)

output = model.generate(
    input_ids,
    max_length=45,
    num_return_sequences=1,
    do_sample=True,
    top_k=8,
    top_p=0.95,
    temperature=0.5,
    repetition_penalty=1.2
)

decoded_output = tokenizer.decode(output[0], skip_special_tokens=True)
print(decoded_output)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Kidney Failure | Muscle weakness, fatigue, irregular heartbeat


In [ ]:
# df.to_dict(orient='records')[0]['Name'] + "|" + df.to_dict(orient='records')[0]['Symptoms']

In [ ]:
# for record in df.to_dict(orient='records'):
#     print(record['Name'] + "|" + record['Symptoms'])

In [ ]:
def tokenize(batch):
  temp =tokenizer(batch['text'], padding=True, truncation=True)
  temp = tokenizer.encode_plus(text, return_tensors='pt', max_length=self.max_length, padding='max_length', truncation=True)
  return temp

print(tokenize(emotion["train"][:5]))

NameError: name 'emotion' is not defined